<h1 align='center'> <b>Data Analysis & Stats</b> </h1>

## ***Librerias***



- <a href="https://pandas.pydata.org/"><img src="https://pandas.pydata.org/docs/_static/pandas.svg" style='height:70px'/></a>
- <a href="https://matplotlib.org/"><img src="https://matplotlib.org/stable/_static/logo2.svg" style='height:50px'/></a>

## ***Languaje***

- <a href="https://www.python.org/"><img src="https://www.python.org/static/img/python-logo.png" style='height:50px'>

## **Manejo de archivos** 

### ***Imports necesarios***

In [ ]:
import pandas as pd     
import matplotlib.pyplot as plt
import numpy as np
import os 

### ***Obtengo la ruta donde se almacena el archivo CSV***

In [ ]:
FOLDER_PATH = os.path.abspath('..')
EVENTS_PATH =  os.path.join(FOLDER_PATH,'src','database','events','events.csv') 
USERS_JSON_PATH = os.path.join(FOLDER_PATH,'src','database','saves','users.json')

### ***Leo el archivo y lo convierto en Data Frame***

In [ ]:
if not os.path.exists(EVENTS_PATH):
    data_frame = pd.DataFrame(columns= ["timestamp","id","evento","cantidad a adivinar","usuarie","estado","texto ingresado","respuesta","nivel"])
    print('No se encontro ningun registro de partidas...')
else:   
    DATA_FRAME = pd.read_csv(EVENTS_PATH,encoding='utf-8',keep_default_na='')

### ***Informacion sobre el Data Frame***

In [ ]:
DATA_FRAME.info()

### ***Tamaño del Data Frame***

- (Filas,Columnas)

In [ ]:
DATA_FRAME.shape

### ***Cosas interesantes***

- Obtener los primeros, por defecto son 5

In [ ]:
DATA_FRAME.head()

- Obtener los ultimos, por defecto son 5

In [ ]:
DATA_FRAME.tail()

## **Empezamos a graficar**

### Porcentaje de partidas finalizadas según género

- ***No quedamos con los datos que nos interesan***

In [ ]:
sub_df = DATA_FRAME.loc[:,["genero","estado"]]

- ***Aplicacamos title case, para no poseer invovenientes a la hora de procesar los generos.***

- ***Filtramos la información de estado para obtener las partidas finalizadas***

In [ ]:
sub_df['genero'] = sub_df['genero'].apply(lambda genero: genero.title())
sub_df = sub_df[sub_df['estado']=='finalizada']

- ***Obtenos los generos y sus respectivos valores***



In [ ]:
cant_x_gender = sub_df['genero'].value_counts()

- **Graficamos**
- Parametros - ***Gráfico tipo Pie***
  - startangle = Define el angulo del grafico en 90°.
  - shadow = Agrega sombra al grafico.
  - bbox_to_anchor = Posiciona fuera del grafico sus respectivas leyendas.
  - wedgeprops = Por medio de un diccionarios personalizamos el grafico.
  - autopct= Agrega el valor porcentual del los datos. 
    (**% = indica que de tipo porcentual**,
    **0.1 = indica la cantidad de decimales agregados**,
    **f = para darle formato porcentul** )

In [ ]:
plt.pie(cant_x_gender.values,
startangle = 90,
shadow = True,
wedgeprops = {'linewidth': 6},
autopct='%1.1f%%')
plt.legend(cant_x_gender.index,title='Generos',bbox_to_anchor=(1.05,1))
plt.axis('equal')
plt.show()

### Porcentaje de partidas finalizadas según el nivel de dificultad

- ***No quedamos con los datos que nos interesan***

In [ ]:
sub_df = DATA_FRAME.loc[:,["nivel","estado"]]

- ***Filtramos la información de estado para obtener las partidas finalizadas***

In [ ]:
sub_df = sub_df[sub_df['estado'] == 'finalizada']

- ***Obtenos los niveles de dificuldad y sus respectivos valores***



In [ ]:
cant_x_difficulty = sub_df['nivel'].value_counts()

- ***Gráfico tipo Pie*** - Parametros 
  - startangle = Define el angulo del grafico en 90°.
  - shadow = Agrega sombra al grafico.
  - bbox_to_anchor = Posiciona fuera del grafico sus respectivas leyendas.
  - wedgeprops = Por medio de un diccionarios personalizamos el grafico.
  - autopct= Agrega el valor porcentual del los datos. 
    (**% = indica que de tipo porcentual**,
    **0.1 = indica la cantidad de decimales agregados**,
    **f = para darle formato porcentul** )

In [ ]:
plt.pie(cant_x_difficulty.values,
startangle = 90,
shadow = True,
wedgeprops = {'linewidth': 6},
autopct='%1.1f%%')#
plt.legend(cant_x_difficulty.index,title='Dificultad',bbox_to_anchor=(1.05,1),)
plt.axis('equal')
plt.show()

### Promedio de tiempo de partida general

In [ ]:
startings = []
endings = []
for partida in DATA_FRAME.values:
    if partida[6] == 'inicio_partida':startings.append(partida[0])
    elif partida[6] == 'fin':endings.append(partida[0])
game_times = []
for start,end in zip(startings,endings):
    game_times.append((end - start)) 
game_times = [(time / 60) for time in game_times]
print(game_times)
mean = round((sum(game_times)/ len(game_times)),2)
print(f'{mean} minutos es el tiempo promedio que dura una partida')

In [ ]:
plt.xlabel('Partidas')
plt.ylabel('Minutos')
plt.title(f'Tiempos de partidas - Promedio: {mean} mins')
plt.plot(game_times,label='Tiempos',marker= "o" if len(tiempos) < 50 else ',')
plt.hlines(mean,0,(len(game_times)-1),label="Promedio",colors='orange')
plt.legend()
plt.show()

### Cantidad de tarjetas en las que el usuarie dió una respuesta errónea

- Obtenenos los estados para cada uno de los usuarios

In [ ]:
sub_df = DATA_FRAME.loc[:,['id','estado','usuarie','correcta']]
sub_df = sub_df[sub_df['estado'] == 'error']
errors_x_user = {user:sub_df[sub_df['usuarie'] == user] for user in sub_df['usuarie'].unique()}
errors_x_user = tuple(((user,len(df.value_counts())) for user,df in errors_x_user.items()))
errors_x_user

In [ ]:
plt.title('Cartas con almenos un error')
plt.xlabel('Cantidad')
plt.ylabel('Usuaries')
for usuarie,cant in errors_x_user :
    plt.bar_label(plt.barh(usuarie,cant,height=0.5,label=f"{usuarie}"),[cant])
plt.axis('equal')
plt.show()

### Cantidad de tarjetas para las que el usuarie no dió respuesta (timeout).

In [ ]:
sub_df = DATA_FRAME.loc[:,['estado','usuarie']]
sub_df = sub_df[sub_df['estado'] == 'timeout']
timeout_por_usuarie = {usuarie:sub_df[sub_df['usuarie'] == usuarie]  for usuarie in sub_df['usuarie'].unique() }

In [ ]:
plt.title('Timeouts por usuaries')
plt.ylabel('Usuarie')
plt.xlabel('Cantidad')
for usuario,df in timeout_por_usuarie.items():
    plt.bar_label(plt.barh(usuario,len(df.index),height=0.5,label=f"{usuario}"),[len(df.index)])
plt.legend()
plt.show()

### Porcentaje de aciertos por usuarie.

In [ ]:
sub_df = DATA_FRAME.loc[:,['usuarie','estado']]
sub_df = sub_df[(sub_df['estado'] == 'error') | (sub_df['estado'] == 'ok')]
intentos_x_usuarie = sub_df.value_counts()
intentos_x_usuarie = {usuarie: tuple(intentos_x_usuarie[usuarie,estado]for estado in sub_df['estado'].unique()) for usuarie in sub_df['usuarie'].unique()}
print(f"Errores y Aciertos por Usuarie - {intentos_x_usuarie}")

In [ ]:
for usuarie,intentos in intentos_x_usuarie.items():
    plt.bar_label(plt.bar(usuarie,(intentos[1]/sum(intentos)*100),width=0.5,label=usuarie,),[str(round(intentos[1]/sum(intentos)*100,2)) + '%'])
plt.title('Aciertos por usuarie')
plt.legend()
plt.yticks([x for x in range(101) if x % 10 == 0])    
plt.show()

### Promedio de tiempo de respuesta para respuestas exitosas.

In [ ]:
sub_df = DATA_FRAME.loc[:,['timestamp','estado','correcta','respuesta']]
tiempos = []
ult_correcta = ''
inicio = 0
ult_timestamp = 0
for intento in sub_df.iloc:
    act_time = intento['timestamp']
    if (intento['correcta'] == intento['respuesta']):
        if (intento['correcta'] != ''):
            tiempos.append(act_time - inicio) 
        inicio = act_time
    elif(intento['correcta'] != ult_correcta):
        inicio = ult_timestamp
    ult_timestamp = act_time
    ult_correcta = intento['correcta']
tiempo_promedio = round(sum(tiempos)/len(tiempos),2)
print(f'{tiempo_promedio = } Segundos')

In [ ]:
plt.plot(tiempos,label='Tiempos',marker= '.' if len(tiempos) < 50 else ',')
plt.hlines(tiempo_promedio,0,(len(tiempos)-1),label="Promedio",colors='orange')
plt.title(f'Tiempos por partidas -  Promedio: {tiempo_promedio} seg.')
plt.legend()
plt.show()

### Tiempo de la respuesta exitosa más rápida.

In [ ]:
sub_df = DATA_FRAME.loc[:,['timestamp','estado','correcta','respuesta']]
mejor_tiempo = 10000.0
ult_correcta = ''
inicio = 0
ult_timestamp = 0
for intento in sub_df.iloc:
    act_time = intento['timestamp']
    if (intento['correcta'] == intento['respuesta']):
        if (intento['correcta'] != ''):
            mejor_tiempo = min(mejor_tiempo, act_time - inicio)
        inicio = act_time
    elif(intento['correcta'] != ult_correcta):
        inicio = ult_timestamp
    ult_timestamp = act_time
    ult_correcta = intento['correcta']
print(f'{mejor_tiempo = }')

In [ ]:
sub_df = DATA_FRAME.loc[:,['timestamp','estado','correcta','respuesta']]